In [1]:
import pandas as pd 
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#df = tuxml.load_dataset()
#path = f"x264/sampledConfigurations_{name_of_technique}_t3.csv"
#lst = ['distBased', 'divDistBased', 'henard', 'random', 'solverBased', 'twise']

path = r"sampledConfigurations_divDistBased_t3.csv"


df = pd.read_csv(path, sep=';')


In [2]:
df

,root,no_asm,no_8x8dct,no_cabac,no_deblock,no_fast_pskip,no_mbtree,no_mixed_refs,no_weightb,rc_lookahead,rc_lookahead_20,rc_lookahead_40,rc_lookahead_60,ref,ref_1,ref_5,ref_9,Performance
0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,268.720000
1,1,1,1,0,1,1,0,1,0,1,1,0,0,1,1,0,0,260.636667
2,1,1,1,1,1,1,1,0,0,1,0,0,1,1,0,0,1,775.810000
3,1,1,1,0,1,1,1,1,0,1,0,0,1,1,0,0,1,651.640000
4,1,1,1,1,1,1,1,1,0,1,0,0,1,1,0,0,1,644.160000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,1,1,1,1,0,0,0,0,1,1,0,0,1,1,0,0,1,581.716667
208,1,1,1,0,0,0,0,1,0,1,0,0,1,1,0,0,1,499.270000
209,1,1,1,0,1,0,0,0,0,1,0,0,1,1,0,0,1,590.550000
210,1,1,1,1,1,0,0,1,0,1,0,0,1,1,0,0,1,493.006667


### Geração de feature importance sem a utilização de hiperparâmetros

In [3]:
def run_featureSelection(df, hyperparameters=None, path=None):
    X = df.drop('Performance', axis=1)
    y = df['Performance']

    if hyperparameters is None:
        rf = RandomForestRegressor()
    else:
        rf = RandomForestRegressor(**hyperparameters)
    
    rf.fit(X, y)

    importances = rf.feature_importances_

    feature_importances = pd.DataFrame(importances, index = X.columns, columns=['importance']).sort_values('importance', ascending=False)

    print(feature_importances)
    
    if hyperparameters is None:
        if path is None:
            feature_importances.to_csv("feature_importance.csv", header=True)
        else:
            feature_importances.to_csv("feature_importance_"+path+".csv", header=True)
    else:
        if path is None:
            feature_importances.to_csv("feature_importanceHP.csv", header=True)
        else:
            feature_importances.to_csv("feature_importanceHP_"+path+".csv", header=True)


run_featureSelection(df)

                 importance
ref_9              0.566026
ref_1              0.173012
ref_5              0.125752
no_fast_pskip      0.051963
no_mixed_refs      0.049198
no_mbtree          0.028693
no_8x8dct          0.004299
no_cabac           0.000474
no_deblock         0.000146
rc_lookahead_20    0.000123
rc_lookahead_60    0.000114
no_weightb         0.000110
rc_lookahead_40    0.000090
ref                0.000000
rc_lookahead       0.000000
no_asm             0.000000
root               0.000000


### Geração de feature importance com a utilização de hiperparâmetros

In [4]:
hyperparams_RF = {
    "max_depth": 25,
    "n_estimators": 10,
    "n_jobs" : -1
}

run_featureSelection(df, hyperparams_RF)

                 importance
ref_9              0.609980
ref_5              0.126211
ref_1              0.125615
no_fast_pskip      0.055055
no_mixed_refs      0.049372
no_mbtree          0.028051
no_8x8dct          0.004350
no_weightb         0.000652
no_cabac           0.000412
rc_lookahead_40    0.000104
rc_lookahead_20    0.000079
rc_lookahead_60    0.000064
no_deblock         0.000054
no_asm             0.000000
rc_lookahead       0.000000
ref                0.000000
root               0.000000


In [5]:
samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']

for config_name in samples_config:
    for t in [1,2,3]:
        path = r"sampledConfigurations_"+config_name+"_t"+str(t)+".csv"
        path_name = config_name+"_t"+str(t)
        df = pd.read_csv(path, sep=';')
        
        run_featureSelection(df, None, path_name)
        run_featureSelection(df, hyperparams_RF, path_name)

                 importance
ref_1              0.462736
ref_9              0.334072
ref_5              0.112207
rc_lookahead_60    0.033703
no_fast_pskip      0.024824
no_deblock         0.010604
rc_lookahead_20    0.008119
no_mbtree          0.007710
no_mixed_refs      0.002427
rc_lookahead_40    0.002383
no_cabac           0.000671
no_8x8dct          0.000543
ref                0.000000
root               0.000000
rc_lookahead       0.000000
no_asm             0.000000
no_weightb         0.000000
                 importance
ref_1              0.580468
ref_9              0.346754
ref_5              0.041437
no_mbtree          0.011204
no_mixed_refs      0.009429
no_fast_pskip      0.003909
no_deblock         0.003453
rc_lookahead_40    0.001315
rc_lookahead_20    0.000791
no_8x8dct          0.000691
no_cabac           0.000442
rc_lookahead_60    0.000106
ref                0.000000
root               0.000000
rc_lookahead       0.000000
no_asm             0.000000
no_weightb         0

                   importance
ref_1            6.853655e-01
no_mbtree        1.110676e-01
ref_9            1.109006e-01
ref_5            3.916542e-02
no_8x8dct        3.828099e-02
no_cabac         1.147938e-02
no_fast_pskip    3.266817e-03
rc_lookahead_40  2.851126e-04
rc_lookahead_20  1.620283e-04
no_deblock       2.150307e-05
rc_lookahead_60  3.165590e-06
no_weightb       1.167288e-06
no_mixed_refs    7.317098e-07
rc_lookahead     0.000000e+00
ref              0.000000e+00
no_asm           0.000000e+00
root             0.000000e+00
                 importance
ref_1              0.650850
ref_9              0.186584
no_mbtree          0.067422
ref_5              0.039326
no_fast_pskip      0.012486
no_8x8dct          0.012291
no_mixed_refs      0.010686
rc_lookahead_60    0.008755
rc_lookahead_40    0.004233
rc_lookahead_20    0.003564
no_cabac           0.001937
no_weightb         0.001857
no_deblock         0.000009
rc_lookahead       0.000000
ref                0.000000
no_asm      

                 importance
ref_1              0.397070
ref_9              0.276134
ref_5              0.139143
no_mixed_refs      0.068761
no_fast_pskip      0.062671
no_mbtree          0.051208
no_8x8dct          0.004141
no_cabac           0.000574
rc_lookahead_40    0.000188
no_deblock         0.000039
rc_lookahead_20    0.000031
no_weightb         0.000025
rc_lookahead_60    0.000015
ref                0.000000
rc_lookahead       0.000000
no_asm             0.000000
root               0.000000
                 importance
ref_1              0.353147
ref_5              0.244765
ref_9              0.209530
no_mixed_refs      0.070441
no_fast_pskip      0.059249
no_mbtree          0.058039
no_8x8dct          0.004211
no_cabac           0.000529
no_deblock         0.000038
rc_lookahead_20    0.000032
rc_lookahead_60    0.000012
rc_lookahead_40    0.000005
no_weightb         0.000002
ref                0.000000
rc_lookahead       0.000000
no_asm             0.000000
root               0